In [1]:
#!/usr/bin/env python
# coding: utf-8

#upload libraries
import pandas as pd
import numpy as np
import json
from os import listdir
from os.path import isfile, join
from itertools import islice
import re
import progressbar
import csv


path = "/Users/ericluo04/Documents/GitHub/Bots-Project/Code/1. HK Dataset/HK English/"

In [16]:
labeledData = pd.read_csv('Data/labeled_tweets.csv', encoding='utf-8')


In [31]:
labeledData = labeledData[labeledData["pro/anti"] != 0]
labeledData["pro/anti"].unique()

array(['0', 'pro/anti', '1'], dtype=object)

In [23]:
pd.crosstab(labeledData, ["pro/anti", "pro/anti"])

ValueError: Shape of passed values is (3, 3), indices imply (538928, 3)

In [12]:
#keywords for pro your topic
hash_pro = list(pd.read_csv("hashtags_pro.txt", sep = ",", names= ['hashtag'], encoding='utf-8').hashtag)
#hash_pro = [''.join(re.findall('[0-9A-Za-z]', hash_pro[i])) for i in range(len(hash_pro))]
keywords_pro = '|'.join(hash_pro)

#keywords for anti your topic
hash_anti = list(pd.read_csv("hashtags_anti.txt", sep = ",", names= ['hashtag'], encoding='utf-8').hashtag)
#hash_anti = [''.join(re.findall('[0-9A-Za-z]', hash_anti[i])) for i in range(len(hash_anti))]
keywords_anti = '|'.join(hash_anti) #for regex 

#get list of files in our directory so we can loop through them 
all_files = [f for f in listdir(path) if isfile(join(path, f))]
if '.DS_Store' in all_files:
    all_files.remove('.DS_Store')

print('files are: ', all_files)

files are:  ['HK English.db', 'tweet.json', 'user_profile.json']


In [14]:
keywords_pro

'StandWithHongKong|FreeHongKong|antiELAB|chinazi|StandwithHK|freedomHK|hkpolicebrutality|freehk|soshk|antiELABhk|HKPoliceTerrorism|FollowBackHongKong|5DemandsNot1Less|chinazism|香港加油|HKHumanRightsandDemocracyAct|boycottMulan|香港人加油|LiberateHongKong|passhkhumanrightsdemocracyact|hongkongpolicebrutality|antichinazi|SaveHongKong|光復香港時代革命|5demandsnotoneless|prayforhongkong|五大訴求缺一不可|光復香港|noextraditiontochina|時代革命|HKGestapo|五大訴求|nochinaextradition|freedomhkg|China70years|hkpoliceterrorist|shout4HK|AntiExtraditionBill|GloryToHongKong|願榮光歸香港|china70yearsofshame|齊上齊落|新屋嶺|科勞手足|沒有暴徒只有暴政|文宣|一個都不能少|合唱團SATB版|管弦樂團|沙田|賀佢老母|死因無可疑|還我真相|fightforhongkong|HKPoliceGangsCollusion|風雨中抱緊自由|🇹🇼|🇭🇰'

In [ ]:
# In[17]:
#seperate tweets into their users
def seperate(tweets):
    '''
    takes as input tweets and seperate them into their own users 
    return a dictionary with keys as user ids and values as description and all tweets of this user in input tweets
    '''
    users = dict()
    bar = progressbar.ProgressBar()
    for twt in bar(tweets):
        id_ = twt['user_id']
        try:
            users[id_]['tweets'].append(twt['text'])
        except:
            u = dict()
            #this user does not exist
            #u['description'] = twt['user']['description']
            u['tweets'] = [twt['text']]
            users[id_] = u
    return(users)

def pro_or_anti(description):
    if bool(re.search(keywords_pro, description, re.IGNORECASE)):
        return(0)
    elif bool(re.search(keywords_anti, description, re.IGNORECASE)):
        return(1)
    else:
        return(None)

def pro_anti(users):
    #list of pro, anti tweets
    pro = []
    anti = []
    bar = progressbar.ProgressBar()
    for usr in bar(users.keys()):
        description = str(users[usr]['tweets'])
        #print(description)
        #see if pro/anti/None
        ind = pro_or_anti(description)
        if ind == 1: #anti
            anti.extend(list(users[usr]['tweets']))
        elif ind == 0: #pro
            pro.extend(list(users[usr]['tweets']))
        else:
            pass
    return(pro, anti)

In [ ]:
#create csv where we will store data

with open('Data/labeled_tweets.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    #wr.writerow(['index', 'tweet', 'label'])

def get_data(int_files, n_max, party = None):
    count = 0
    n_tweets = 0
    for file in int_files:
        path_file = path + file
        n = 5000  # Or whatever chunk size you want
        with open(path_file, 'r') as f:
            pro, anti = [], []
            for tweets in iter(lambda: list(islice((json.loads(line) for line in f), n)), []):
                users = seperate(tweets)
                pro_, anti_ = pro_anti(users) #this gives us a list of pro and anti tweets
                pro.extend(pro_)
                anti.extend(anti_)
                count += 1
                #save every 10 chunks of data (i.e. 50000 tweets)
                if count % 1 == 0:
                    pro = [[i, 0] for i in pro_]
                    anti = [[i, 1] for i in anti_]
                    if party == 'pro':
                        all_ = pro
                    elif party == 'anti':
                        all_ = anti
                    else:
                        all_ = pro + anti
                    df = pd.DataFrame(all_, columns = ['tweet', 'pro/anti'])
                    n_tweets += df.shape[0]
                    print(n_tweets)
                    #print(df.head(10))
                    with open('Data/labeled_tweets.csv', 'a',  encoding = 'utf-8') as f_data:
                        df.to_csv(f_data, header=True)
                if n_tweets > n_max:  
                    print('stop: maximum number of tweets reached')
                    break

n_max = 100000 #max_number of tweets
int_files = [all_files[1]]
get_data(int_files, n_max)

In [28]:
#!/usr/bin/env python
# coding: utf-8

#upload libraries
import pandas as pd
import numpy as np
import json
import sqlite3
from os import listdir
from os.path import isfile, join
from itertools import islice
import re
import progressbar
import csv

In [29]:
DB = r"C:\Users\ericluo04\Documents\GitHub\Bots-Project\Code\1. HK Dataset\HK English\HK English.db"

def get_all_users( json_str = False ):
    conn = sqlite3.connect( DB )
    conn.row_factory = sqlite3.Row # This enables column access by name: row['column_name'] 
    db = conn.cursor()

    rows = db.execute('''
    SELECT * from tweet
    ''').fetchall()

    conn.commit()
    conn.close()

    if json_str:
        return [dict(ix) for ix in rows] #CREATE JSON

    return rows

def get_all_descriptions( json_str = False ):
    conn = sqlite3.connect( DB )
    conn.row_factory = sqlite3.Row # This enables column access by name: row['column_name'] 
    db = conn.cursor()

    rows = db.execute('''
    SELECT * from user_profile
    ''').fetchall()

    conn.commit()
    conn.close()

    if json_str:
        return [dict(ix) for ix in rows] #CREATE JSON

    return rows

twtdata = get_all_users(json_str = True)
twtdesc = get_all_descriptions(json_str = True)

In [35]:
twtdesc

[{'user_id': 1081,
  'screen_name': 'davemcclure',
  'name': 'Dave McClure ',
  'created_at': '2006-07-16 00:37:30',
  'description': 'Founder @500Startups. Investor 10 Unicorns, 5 IPOs (TWLO SEND LYFT REAL Life360). Forbes Midas Top 100 VCs (2016, 2017). PayPal, Founders Fund, JHU. ',
  'status_id': 1180879547895386112,
  'geo_enabled': 'True',
  'protected': 'False',
  'friends_count': 18353,
  'followers_count': 359693,
  'favourites_count': 46563,
  'statuses_count': 76756,
  'lang': None,
  'location': 'Silicon Valley ',
  'verified': 'True',
  'profile_url': None,
  'default_image': 'False',
  'time_zone': None,
  'UTC_offset': None,
  'profile_pic_hash': None,
  'profile_pic_filename': None,
  'profile_banner_hash': None,
  'profile_banner_filename': None,
  'obtained': '2019-10-06 00:00:00',
  'last_accessed': '2019-10-06 00:00:00',
  'active': 'True'},
 {'user_id': 1427,
  'screen_name': 'tchambers',
  'name': 'Tim Chambers ',
  'created_at': '2006-07-16 17:19:25',
  'descript

In [33]:
twtdesc['description' == 'chinazi']

{'user_id': 1081,
 'screen_name': 'davemcclure',
 'name': 'Dave McClure ',
 'created_at': '2006-07-16 00:37:30',
 'description': 'Founder @500Startups. Investor 10 Unicorns, 5 IPOs (TWLO SEND LYFT REAL Life360). Forbes Midas Top 100 VCs (2016, 2017). PayPal, Founders Fund, JHU. ',
 'status_id': 1180879547895386112,
 'geo_enabled': 'True',
 'protected': 'False',
 'friends_count': 18353,
 'followers_count': 359693,
 'favourites_count': 46563,
 'statuses_count': 76756,
 'lang': None,
 'location': 'Silicon Valley ',
 'verified': 'True',
 'profile_url': None,
 'default_image': 'False',
 'time_zone': None,
 'UTC_offset': None,
 'profile_pic_hash': None,
 'profile_pic_filename': None,
 'profile_banner_hash': None,
 'profile_banner_filename': None,
 'obtained': '2019-10-06 00:00:00',
 'last_accessed': '2019-10-06 00:00:00',
 'active': 'True'}

In [25]:
# In[17]:
#seperate tweets into their users
def seperate(tweets, twtdesc):
    '''
    takes as input tweets and seperate them into their own users 
    return a dictionary with keys as user ids and values as description and all tweets of this user in input tweets
    '''
    users = dict()
    bar = progressbar.ProgressBar()
    for twt in bar(tweets):
        id_ = twt['user_id']
        try:
            users[id_]['tweets'].append(twt['text'])
        except:
            u = dict()
            #this user does not exist
            u['description'] = twtdesc['user_id' == id_]['description']
            u['tweets'] = [twt['text']]
            users[id_] = u
    return(users)

In [8]:
#keywords for pro your topic
hash_pro = list(pd.read_csv("hashtags_pro.txt", sep = ",", skiprows= list(range(9)), names= ['hashtag'], encoding='utf-8').hashtag)
#hash_pro = [''.join(re.findall('[0-9A-Za-z]', hash_pro[i])) for i in range(len(hash_pro))]
keywords_pro = '|'.join(hash_pro)

#keywords for anti your topic
hash_anti = list(pd.read_csv("hashtags_anti.txt", sep = ",", skiprows= list(range(9)), names= ['hashtag'], encoding='utf-8').hashtag)
#hash_anti = [''.join(re.findall('[0-9A-Za-z]', hash_anti[i])) for i in range(len(hash_anti))]
keywords_anti = '|'.join(hash_anti) #for regex 

def pro_or_anti(description):
    if bool(re.search(keywords_pro, description, re.IGNORECASE)):
        return(0)
    elif bool(re.search(keywords_anti, description, re.IGNORECASE)):
        return(1)
    else:
        return(None)

def pro_anti(users):
    #list of pro, anti tweets
    pro = []
    anti = []
    bar = progressbar.ProgressBar()
    for usr in bar(users.keys()):
        description = str(users[usr]['description'])
        #print(description)
        #see if pro/anti/None
        ind = pro_or_anti(description)
        if ind == 1: #antiocrat
            anti.extend(list(users[usr]['tweets']))
        elif ind == 0: #proublican
            pro.extend(list(users[usr]['tweets']))
        else:
            pass
    return(pro, anti)


In [11]:
#create csv where we will store data

with open('Data/labeled_tweets.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    #wr.writerow(['index', 'tweet', 'label'])

In [26]:
def get_data(twtdata, n_max, party = None):
    count = 0
    n_tweets = 0
    n = 5000  # Or whatever chunk size you want
    pro, anti = [], []

    #for tweets in iter(lambda: list(islice((json.loads(json.dumps(users))), n)), []):   
    for i in range(0, len(twtdata), n):
        tweets = json.loads(json.dumps(twtdata[i:i+n]))
        users = seperate(tweets, twtdesc)
        pro_, anti_ = pro_anti(users) #this gives us a list of pro and anti tweets
        pro.extend(pro_)
        anti.extend(anti_)
        count += 1
        #save every 10 chunks of data (i.e. 50000 tweets)
        if count % 10 == 0:
            pro = [[i, 0] for i in pro_]
            anti = [[i, 1] for i in anti_]
            if party == 'pro':
                all_ = pro
            elif party == 'anti':
                all_ = anti
            else:
                all_ = pro + anti
            df = pd.DataFrame(all_, columns = ['tweet', 'pro/anti'])
            n_tweets += df.shape[0]
            print(n_tweets)
            #print(df.head(10))
            with open('Data/labeled_tweets.csv', 'a',  encoding = 'utf-8') as f_data:
                df.to_csv(f_data, header=True)
        if n_tweets > n_max:  
            print('stop: maximum number of tweets reached')
            break 

In [27]:
n_max = 100000000 #max_number of tweets
get_data(twtdata, n_max)

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

0


100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

0


100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

...

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

0


100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

0


100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################